# Whisper Fine-Tuning on LibriSpeech with Optuna

This notebook is modifed from Fine-Tune Whisper For Multilingual ASR with 🤗 Transformers Colab created by Sanchit Gandhi avalible at https://colab.research.google.com/github/sanchit-gandhi/notebooks/blob/main/fine_tune_whisper.ipynb. Our work is only the modifications to the original notebook.

In [ ]:
# use datasets to download and prepare our training data and transformers to load and train our Whisper model.
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install optuna


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-4eegb0gf
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-4eegb0gf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 7.6 MB 31.0 MB/s 
  Created wheel for transformers: filename=transformers-4.26.0.dev0-py3-none-any.whl size=5881991 sha256=7046b10cf4f764a4ee2d0060f559fcd330fd6eccb00d48d96d79c8e19a604f10
  Stored in directory: /tmp/pip-ephem-wheel-cache-f5ezb4ua/wheels/42/68/45/c63edff61c292f2dfd4df4ef6522dcbecc603e7af82813c1d7
Successfully built transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
   

In [ ]:
# import the relavant libraries for loggin in
from huggingface_hub import HfApi, HfFolder

# set api for login and save token\
api=HfApi()
api.set_access_token(token)
folder = HfFolder()
folder.save_token(token)

/usr/local/lib/python3.8/dist-packages/huggingface_hub/utils/_deprecation.py:128: FutureWarning: 'set_access_token' (from 'huggingface_hub.hf_api') is deprecated and will be removed from version '0.14'. `HfApi.set_access_token` is deprecated as it is very ambiguous. Use `login` or `set_git_credential` instead.
  warnings.warn(warning_message, FutureWarning)
/usr/local/lib/python3.8/dist-packages/huggingface_hub/utils/_deprecation.py:128: FutureWarning: 'write_to_credential_store' (from 'huggingface_hub.utils._git_credential') is deprecated and will be removed from version '0.14'. Please use `huggingface_hub.set_git_credential` instead as it allows the user to chose which git-credential tool to use.
  warnings.warn(warning_message, FutureWarning)


In [ ]:
import pickle
from datasets import Audio
from datasets import Dataset
from datasets import Features

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny.en")

Downloading:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny.en", language="English", task="transcribe")

Downloading:   0%|          | 0.00/844 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/999k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-tiny.en", language="English", task="transcribe")

In [ ]:
from datasets import load_dataset
from datasets import DownloadConfig
# link = "DTU54DL/commo-test1k-whisper-proc"
ds_train = load_dataset(r"CristianaLazar/librispeech5k_train", download_config=DownloadConfig(delete_extracted=True))
ds_test = load_dataset(r"CristianaLazar/librispeech_augm_validation-tiny", download_config=DownloadConfig(delete_extracted=True))

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train.360 split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/CristianaLazar___parquet/CristianaLazar--librispeech5k_train-d689137cca331e69/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
train = ds_train["train.360"].select(range(0, 500))
test = ds_test["validation"].select(range(0, 100))

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch
        
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import evaluate

metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
def optuna_hp_space(trial):

    return {

        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0.1, 0.2, log=True),

        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8]),

    }

In [ ]:
# load the pre-trained Whisper small checkpoint.
from transformers import WhisperForConditionalGeneration

def model_init(trial):
    model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en")
    model.config.forced_decoder_ids = None
    model.config.suppress_tokens = []

    return model

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/whisper-tiny-libri_search",  # change to a repo name of your choice
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    warmup_steps=2,
    max_steps=500,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=50,
    eval_steps=50,
    logging_steps=50,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

PyTorch: setting up devices


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=None,
    train_dataset=train,
    eval_dataset=test,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    model_init=model_init,
    
)

/whisper-tiny-libri_search is already a clone of https://huggingface.co/CristianaLazar/whisper-tiny-libri_search. Make sure you pull the latest changes with `repo.git_pull()`.
max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend


In [ ]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=4,
)

`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
50,0.904800,0.747488,28.880266
100,0.256700,0.685584,25.332594
150,0.128400,0.689597,23.946785
200,0.058200,0.682746,24.223947
250,0.030100,0.720203,24.390244
300,0.016700,0.705789,24.168514
350,0.009800,0.707749,22.893570
400,0.005600,0.697938,23.004435
450,0.004600,0.705192,22.671840
500,0.003800,0.705268,22.671840


`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient c

Step,Training Loss,Validation Loss,Wer
50,0.927200,0.790834,27.549889
100,0.310300,0.645642,24.057650
150,0.162200,0.638435,24.057650
200,0.082300,0.637445,23.392461
250,0.044300,0.635826,23.281596
300,0.022500,0.648954,24.501109
350,0.016300,0.657079,24.279379
400,0.011500,0.661503,24.778271
450,0.009500,0.665024,24.279379
500,0.008300,0.665801,24.611973


`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient c

Step,Training Loss,Validation Loss,Wer
50,0.868700,0.734508,27.494457
100,0.259100,0.661372,25.388027
150,0.131500,0.672508,25.277162
200,0.057600,0.663650,24.279379
250,0.029100,0.667490,23.946785
300,0.016200,0.672271,23.115299
350,0.010500,0.681922,23.226164
400,0.006500,0.685979,22.893570
450,0.005600,0.689046,22.893570
500,0.004800,0.690212,23.004435


`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient c

Step,Training Loss,Validation Loss,Wer
50,0.936300,0.801701,28.991131
100,0.321500,0.643573,23.946785
150,0.168800,0.636789,23.558758
200,0.088400,0.634394,24.002217
250,0.047400,0.635364,23.614191
300,0.024900,0.646138,24.390244
350,0.017400,0.655878,23.835920
400,0.012500,0.660260,24.889135
450,0.010400,0.663743,24.889135
500,0.009000,0.664997,25.110865


`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient c